In [343]:
#pip install sparqlwrapper

In [344]:
import numpy as np
import pandas as pd
import json
from SPARQLWrapper import SPARQLWrapper, JSON

In [345]:
wds = "https://query.wikidata.org/sparql" 

In [346]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql", 
            agent="Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11")

In [347]:
def get_sparql_dataframe(service, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    sparql = SPARQLWrapper(service,  agent="Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11")
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

In [348]:
# From https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries/examples#Cats
rq = """
SELECT 
?humanLabel 
?human 
?country 
?occup 
?gender 
?fatherLabel 
?father 
?motherLabel 
?mother 
?siblingLabel
?sibling 


WHERE
{
  
?human wdt:P31 wd:Q5

{ { ?human wdt:P106 wd:010800557 } UNION {?human wdt:P106 wd:Q33999} UNION {?human wdt:P106 wd:Q177220} }.

{ { ?human wdt:P27 wd:Q30 } UNION {?human wdt:P27 wd:Q668} }

?human wdt:P27 ?country.

?human wdt:P106 ?occup 
 OPTIONAL{?human wdt:P21 ?gender .} 
 OPTIONAL{?human wdt:P22 ?father .}
 #OPTIONAL{?father wdt: P106 ?fatheroccup .}
 OPTIONAL{?human wdt:P25 ?mother .}
 #OPTIONAL{?mother wdt:P106 ?motheroccup .}
 OPTIONAL { ?human wdt:P3373 ?sibling .}
 #OPTIONAL {?sibling wdt:P106 ?siblingoccup .}

SERVICE wikibase:label  {bd:serviceParam wikibase:language "en" .
                         }   

filter (?occup = wd:Q10800557 || ?occup = wd:Q33999 || ?occup = wd:Q177220)
filter (?country = wd:Q30 || ?country = wd:Q668)
}
"""
#sparql.setReturnFormat(JSON)
#results = sparql.query().convert()

In [351]:
df = get_sparql_dataframe(wds, rq)

In [352]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81371 entries, 0 to 81370
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   humanLabel    81371 non-null  object
 1   human         81371 non-null  object
 2   country       81371 non-null  object
 3   occup         81371 non-null  object
 4   gender        81361 non-null  object
 5   fatherLabel   6347 non-null   object
 6   father        6347 non-null   object
 7   motherLabel   4686 non-null   object
 8   mother        4686 non-null   object
 9   siblingLabel  7286 non-null   object
 10  sibling       7286 non-null   object
dtypes: object(11)
memory usage: 6.8+ MB


In [353]:
df[df['humanLabel'] == "Elvis Presley"]

,humanLabel,human,country,occup,gender,fatherLabel,father,motherLabel,mother,siblingLabel,sibling
0,Elvis Presley,http://www.wikidata.org/entity/Q303,http://www.wikidata.org/entity/Q30,http://www.wikidata.org/entity/Q33999,http://www.wikidata.org/entity/Q6581097,Vernon Elvis Presley,http://www.wikidata.org/entity/Q4377426,Gladys Presley,http://www.wikidata.org/entity/Q12363015,Jessie Garon Presley,http://www.wikidata.org/entity/Q16408125
1,Elvis Presley,http://www.wikidata.org/entity/Q303,http://www.wikidata.org/entity/Q30,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q6581097,Vernon Elvis Presley,http://www.wikidata.org/entity/Q4377426,Gladys Presley,http://www.wikidata.org/entity/Q12363015,Jessie Garon Presley,http://www.wikidata.org/entity/Q16408125
2,Elvis Presley,http://www.wikidata.org/entity/Q303,http://www.wikidata.org/entity/Q30,http://www.wikidata.org/entity/Q10800557,http://www.wikidata.org/entity/Q6581097,Vernon Elvis Presley,http://www.wikidata.org/entity/Q4377426,Gladys Presley,http://www.wikidata.org/entity/Q12363015,Jessie Garon Presley,http://www.wikidata.org/entity/Q16408125
250,Elvis Presley,http://www.wikidata.org/entity/Q303,http://www.wikidata.org/entity/Q30,http://www.wikidata.org/entity/Q33999,http://www.wikidata.org/entity/Q6581097,Vernon Elvis Presley,http://www.wikidata.org/entity/Q4377426,Gladys Presley,http://www.wikidata.org/entity/Q12363015,Jessie Garon Presley,http://www.wikidata.org/entity/Q16408125
251,Elvis Presley,http://www.wikidata.org/entity/Q303,http://www.wikidata.org/entity/Q30,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q6581097,Vernon Elvis Presley,http://www.wikidata.org/entity/Q4377426,Gladys Presley,http://www.wikidata.org/entity/Q12363015,Jessie Garon Presley,http://www.wikidata.org/entity/Q16408125
252,Elvis Presley,http://www.wikidata.org/entity/Q303,http://www.wikidata.org/entity/Q30,http://www.wikidata.org/entity/Q10800557,http://www.wikidata.org/entity/Q6581097,Vernon Elvis Presley,http://www.wikidata.org/entity/Q4377426,Gladys Presley,http://www.wikidata.org/entity/Q12363015,Jessie Garon Presley,http://www.wikidata.org/entity/Q16408125


In [354]:
df['occup'].value_counts()

http://www.wikidata.org/entity/Q33999       44111
http://www.wikidata.org/entity/Q177220      20211
http://www.wikidata.org/entity/Q10800557    17049
Name: occup, dtype: int64

In [355]:
df_base = df.drop_duplicates(subset='human', keep='first')

In [356]:
df_base.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51978 entries, 0 to 81370
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   humanLabel    51978 non-null  object
 1   human         51978 non-null  object
 2   country       51978 non-null  object
 3   occup         51978 non-null  object
 4   gender        51968 non-null  object
 5   fatherLabel   2383 non-null   object
 6   father        2383 non-null   object
 7   motherLabel   1573 non-null   object
 8   mother        1573 non-null   object
 9   siblingLabel  2489 non-null   object
 10  sibling       2489 non-null   object
dtypes: object(11)
memory usage: 4.8+ MB


In [357]:
df_base.head()

,humanLabel,human,country,occup,gender,fatherLabel,father,motherLabel,mother,siblingLabel,sibling
0,Elvis Presley,http://www.wikidata.org/entity/Q303,http://www.wikidata.org/entity/Q30,http://www.wikidata.org/entity/Q33999,http://www.wikidata.org/entity/Q6581097,Vernon Elvis Presley,http://www.wikidata.org/entity/Q4377426,Gladys Presley,http://www.wikidata.org/entity/Q12363015,Jessie Garon Presley,http://www.wikidata.org/entity/Q16408125
3,Bob Dylan,http://www.wikidata.org/entity/Q392,http://www.wikidata.org/entity/Q30,http://www.wikidata.org/entity/Q33999,http://www.wikidata.org/entity/Q6581097,None,None,None,None,None,None
6,Patti Smith,http://www.wikidata.org/entity/Q557,http://www.wikidata.org/entity/Q30,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q6581072,None,None,None,None,None,None
7,Neil Young,http://www.wikidata.org/entity/Q633,http://www.wikidata.org/entity/Q30,http://www.wikidata.org/entity/Q33999,http://www.wikidata.org/entity/Q6581097,Scott Young,http://www.wikidata.org/entity/Q1780491,None,None,Astrid Young,http://www.wikidata.org/entity/Q4811453
10,Stevie Wonder,http://www.wikidata.org/entity/Q714,http://www.wikidata.org/entity/Q30,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q6581097,None,None,Lula Mae Hardaway,http://www.wikidata.org/entity/Q6702757,None,None


In [358]:
df['gender'].value_counts()

http://www.wikidata.org/entity/Q6581097     42235
http://www.wikidata.org/entity/Q6581072     38943
http://www.wikidata.org/entity/Q1052281       116
http://www.wikidata.org/entity/Q48270          34
http://www.wikidata.org/entity/Q2449503        14
http://www.wikidata.org/entity/Q15145779        8
http://www.wikidata.org/entity/Q1097630         4
http://www.wikidata.org/entity/Q18116794        3
t1937007943                                     1
http://www.wikidata.org/entity/Q301702          1
http://www.wikidata.org/entity/Q15145778        1
http://www.wikidata.org/entity/Q12964198        1
Name: gender, dtype: int64

In [359]:
df_base['countryname'] = np.where(df_base['country']=='http://www.wikidata.org/entity/Q30', 'American', 'Indian')

C:\Users\suvad\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [360]:
df_base.loc[df_base['gender'] == 'http://www.wikidata.org/entity/Q6581097', 'gendername'] = 'Male'  
df_base.loc[df_base['gender'] == 'http://www.wikidata.org/entity/Q6581072', 'gendername'] = 'Female' 
df_base.loc[(df_base['gender'] != 'http://www.wikidata.org/entity/Q6581097') & 
            (df_base['gender'] != 'http://www.wikidata.org/entity/Q6581072'), 'gendername'] = 'Unknown'  

C:\Users\suvad\anaconda3\lib\site-packages\pandas\core\indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\suvad\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
C:\Users\suvad\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas

In [361]:
df_base['gendername'].value_counts()

Male       29356
Female     22503
Unknown      119
Name: gendername, dtype: int64

In [362]:
df_base = df_base[["humanLabel", "human", "countryname", "gendername"]]

In [363]:
df_base['countryname'].value_counts()

American    44715
Indian       7263
Name: countryname, dtype: int64

In [364]:
df_base.head()

,humanLabel,human,countryname,gendername
0,Elvis Presley,http://www.wikidata.org/entity/Q303,American,Male
3,Bob Dylan,http://www.wikidata.org/entity/Q392,American,Male
6,Patti Smith,http://www.wikidata.org/entity/Q557,American,Female
7,Neil Young,http://www.wikidata.org/entity/Q633,American,Male
10,Stevie Wonder,http://www.wikidata.org/entity/Q714,American,Male


In [365]:
rq1 = """
SELECT

?human ?humanLabel ?fatherLabel ?father ?fatheroccupLabel

WHERE

{
?human wdt:P31 wd:Q5

{ { ?human wdt:P106 wd:Q10800557 } UNION {?human wdt:P106 wd:Q33999} UNION {?human wdt:P106 wd:Q177220} } .

{ { ?human wdt:P27 wd:Q30 } UNION {?human wdt:P27 wd:Q668 } } .

?human wdt:P22 ?father .
?father wdt:P106 ?fatheroccup .

SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
}
"""

In [366]:
df_father = get_sparql_dataframe(wds, rq1)

In [367]:
df_father.head()

,human,humanLabel,fatherLabel,father,fatheroccupLabel
0,http://www.wikidata.org/entity/Q303,Elvis Presley,Vernon Elvis Presley,http://www.wikidata.org/entity/Q4377426,actor
1,http://www.wikidata.org/entity/Q303,Elvis Presley,Vernon Elvis Presley,http://www.wikidata.org/entity/Q4377426,farmer
2,http://www.wikidata.org/entity/Q303,Elvis Presley,Vernon Elvis Presley,http://www.wikidata.org/entity/Q4377426,chauffeur
3,http://www.wikidata.org/entity/Q303,Elvis Presley,Vernon Elvis Presley,http://www.wikidata.org/entity/Q4377426,painter
4,http://www.wikidata.org/entity/Q633,Neil Young,Scott Young,http://www.wikidata.org/entity/Q1780491,biographer


In [368]:
rq2 = """
SELECT

?human ?humanLabel ?motherLabel ?mother ?motheroccupLabel 

WHERE

{
?human wdt:P31 wd:Q5

{ { ?human wdt:P106 wd:Q10800557 } UNION {?human wdt:P106 wd:Q33999} UNION {?human wdt:P106 wd:Q177220} } .

{ { ?human wdt:P27 wd:Q30 } UNION {?human wdt:P27 wd:Q668 } } .

?human wdt:P25 ?mother .
?mother wdt:P106 ?motheroccup .

  
SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
}
"""

In [369]:
df_mother = get_sparql_dataframe(wds, rq2)
df_mother.head()

,human,humanLabel,motherLabel,mother,motheroccupLabel
0,http://www.wikidata.org/entity/Q48622,Esha Deol,Hema Malini,http://www.wikidata.org/entity/Q33635,politician
1,http://www.wikidata.org/entity/Q48622,Esha Deol,Hema Malini,http://www.wikidata.org/entity/Q33635,film director
2,http://www.wikidata.org/entity/Q48622,Esha Deol,Hema Malini,http://www.wikidata.org/entity/Q33635,film producer
3,http://www.wikidata.org/entity/Q48622,Esha Deol,Hema Malini,http://www.wikidata.org/entity/Q33635,dancer
4,http://www.wikidata.org/entity/Q48622,Esha Deol,Hema Malini,http://www.wikidata.org/entity/Q33635,television actor


In [370]:
rq3 = """
SELECT

?human ?humanLabel ?siblingLabel ?sibling ?siblingoccupLabel 

WHERE

{
?human wdt:P31 wd:Q5

{ { ?human wdt:P106 wd:Q10800557 } UNION {?human wdt:P106 wd:Q33999} UNION {?human wdt:P106 wd:Q177220} } .

{ { ?human wdt:P27 wd:Q30 } UNION {?human wdt:P27 wd:Q668 } } .

?human wdt:P3373 ?sibling .
?sibling wdt:P106 ?siblingoccup .

  
SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
}
"""

In [371]:
df_sibling = get_sparql_dataframe(wds, rq3)
df_sibling.head()

,human,humanLabel,siblingLabel,sibling,siblingoccupLabel
0,http://www.wikidata.org/entity/Q44855,Jackie Jackson,Michael Jackson,http://www.wikidata.org/entity/Q2831,screenwriter
1,http://www.wikidata.org/entity/Q44855,Jackie Jackson,Michael Jackson,http://www.wikidata.org/entity/Q2831,actor
2,http://www.wikidata.org/entity/Q44855,Jackie Jackson,Michael Jackson,http://www.wikidata.org/entity/Q2831,businessperson
3,http://www.wikidata.org/entity/Q44855,Jackie Jackson,Michael Jackson,http://www.wikidata.org/entity/Q2831,poet
4,http://www.wikidata.org/entity/Q44855,Jackie Jackson,Michael Jackson,http://www.wikidata.org/entity/Q2831,entrepreneur


In [372]:
from pandas import DataFrame
auxiliaryList = []
for word in df_father[["fatheroccupLabel"]].values:
    if word not in auxiliaryList:
        auxiliaryList.append(word)
for word in df_mother[["motheroccupLabel"]].values:
    if word not in auxiliaryList:
        auxiliaryList.append(word)
for word in df_sibling[["siblingoccupLabel"]].values:
    if word not in auxiliaryList:
        auxiliaryList.append(word)
df_occup = DataFrame(auxiliaryList)

In [373]:
df_occup.to_csv('occup.csv')

In [374]:
lst_actor = ['screenwriter', 'actor', 'voice actor', 'film producer', 'television actor', 'costume designer', 
             'film director', 'theater director', 'film actor', 'composer', 'cinematographer', 'director', 
             'television producer', 'stage actor', 'film score composer', 'lyricist', 'television director', 
             'stunt performer', 'stunt coordinator', 'camera operator', 'film editor', 'producer', 'filmmaker', 
             'child actor', 'choreographer', 'dub actor', 'make-up artist', 'theatrical producer', 'film critic',
             'dialogue writer', 'documentary filmmaker', 'playback singer', 'theatre designer', 'casting director',
             'theatre manager', 'music video director', 'pornographic film producer', 'performance artist', 
             'film studio executive', 'pornographic actor', 'set decorator', 'silent film actor', 'entertainer', 'acting',
             'script doctor', 'film screenwriter', 'television writer', 'musical theatre actor', 'creative director',
             'assistant director', 'Set dresser', 'dubbing director']
lst_singer = ['singer', 'opera singer', 'composer', 'record producer', 'musician', 'music executive', 'singer-songwriter',
                'guitarist', 'songwriter', 'jazz musician', 'percussionist', 'film score composer', 'music producer', 
                'jazz guitarist', 'saxophonist', 'music director', 'lyricist', 'vocal coach', 'music arranger', 'trumpeter', 
                'recording artist', 'drummer', 'bandleader', 'multi-instrumentalist', 'bass-guitarist', 'Song-plugger', 
                'mandolinist', 'country musician', 'trombonist', 'session musician', 'bassist', 'music publisher', 'violinist',
                'musicologist', 'keyboardist', 'vocalist', 'playback singer', 'jazz pianist', 'music video director', 
                'salsa musician', 'rock guitarist', 'ghost singer', 'choir director', 'jazz singer', 'music interpreter', 
                'classical guitarist', 'child singer', 'lead vocalist', 'soul singer', 'musical theatre actor', 
                'music promoter',
                'music artist', 'jazz bassist', 'music management']

occup_actor = ['http://www.wikidata.org/entity/Q33999', 'http://www.wikidata.org/entity/Q10800557']
occup_singer = ['http://www.wikidata.org/entity/Q177220']

In [375]:
df_base = df_base.assign(father_actor=df_base.human.isin(df_father[df_father['fatheroccupLabel'].isin(lst_actor)].human).astype(int))
df_base = df_base.assign(mother_actor=df_base.human.isin(df_mother[df_mother['motheroccupLabel'].isin(lst_actor)].human).astype(int))
df_base = df_base.assign(sibling_actor=df_base.human.isin(df_sibling[df_sibling['siblingoccupLabel'].isin(lst_actor)].human).astype(int))

In [376]:
df_base[df_base["father_actor"] == 1].head()

,humanLabel,human,countryname,gendername,father_actor,mother_actor,sibling_actor
0,Elvis Presley,http://www.wikidata.org/entity/Q303,American,Male,1,0,0
60,Corbin Bleu,http://www.wikidata.org/entity/Q4617,American,Male,1,0,0
99,Beck,http://www.wikidata.org/entity/Q11901,American,Male,1,1,0
108,Asha Bhosle,http://www.wikidata.org/entity/Q38393,Indian,Female,1,0,1
134,Salman Khan,http://www.wikidata.org/entity/Q9543,Indian,Male,1,0,1


In [377]:
df_base['father_actor'].value_counts()

0    50430
1     1548
Name: father_actor, dtype: int64

In [378]:
df_base['mother_actor'].value_counts()

0    51037
1      941
Name: mother_actor, dtype: int64

In [379]:
df_base['sibling_actor'].value_counts()

0    49943
1     2035
Name: sibling_actor, dtype: int64

In [380]:
df_base = df_base.assign(father_singer=df_base.human.isin(df_father[df_father['fatheroccupLabel'].isin(lst_singer)].human).astype(int))
df_base = df_base.assign(mother_singer=df_base.human.isin(df_mother[df_mother['motheroccupLabel'].isin(lst_singer)].human).astype(int))
df_base = df_base.assign(sibling_singer=df_base.human.isin(df_sibling[df_sibling['siblingoccupLabel'].isin(lst_singer)].human).astype(int))

In [381]:
df_base[df_base["father_singer"] == 1].head()

,humanLabel,human,countryname,gendername,father_actor,mother_actor,sibling_actor,father_singer,mother_singer,sibling_singer
84,Prince,http://www.wikidata.org/entity/Q7542,American,Male,0,0,0,1,1,1
99,Beck,http://www.wikidata.org/entity/Q11901,American,Male,1,1,0,1,0,0
108,Asha Bhosle,http://www.wikidata.org/entity/Q38393,Indian,Female,1,0,1,1,0,1
295,Michael Jackson,http://www.wikidata.org/entity/Q2831,American,Male,1,0,1,1,0,1
312,Quentin Tarantino,http://www.wikidata.org/entity/Q3772,American,Male,1,0,0,1,0,0


In [382]:
df_base = df_base.assign(is_actor=df_base.human.isin(df[df['occup'].isin(occup_actor)].human).astype(int))

In [383]:
df_base = df_base.assign(is_singer=df_base.human.isin(df[df['occup'].isin(occup_singer)].human).astype(int))

In [384]:
df_base.head()

,humanLabel,human,countryname,gendername,father_actor,mother_actor,sibling_actor,father_singer,mother_singer,sibling_singer,is_actor,is_singer
0,Elvis Presley,http://www.wikidata.org/entity/Q303,American,Male,1,0,0,0,0,0,1,1
3,Bob Dylan,http://www.wikidata.org/entity/Q392,American,Male,0,0,0,0,0,0,1,1
6,Patti Smith,http://www.wikidata.org/entity/Q557,American,Female,0,0,0,0,0,0,0,1
7,Neil Young,http://www.wikidata.org/entity/Q633,American,Male,0,0,0,0,0,1,1,1
10,Stevie Wonder,http://www.wikidata.org/entity/Q714,American,Male,0,0,0,0,1,0,0,1


In [385]:
df_base['is_actor'].value_counts()

1    39832
0    12146
Name: is_actor, dtype: int64

In [386]:
df_base['is_singer'].value_counts()

0    35949
1    16029
Name: is_singer, dtype: int64

In [387]:
rq4 = """
SELECT ?human ?humanLabel ?awardLabel

WHERE

{
?human wdt:P31 wd:Q5

{ { ?human wdt:P106 wd:Q10800557 } UNION {?human wdt:P106 wd:Q33999} UNION {?human wdt:P106 wd:Q177220} } .

{ { ?human wdt:P27 wd:Q30 } UNION {?human wdt:P27 wd:Q668 } } .

?human wdt:P166 ?award.

SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }

}
"""

In [388]:
df_awards = get_sparql_dataframe(wds, rq4)
df_awards.head()

,human,humanLabel,awardLabel
0,http://www.wikidata.org/entity/Q40504,Jim Carrey,Canada's Walk of Fame
1,http://www.wikidata.org/entity/Q40504,Jim Carrey,MTV Movie Award for Best Villain
2,http://www.wikidata.org/entity/Q40504,Jim Carrey,Chevalier des Arts et des Lettres
3,http://www.wikidata.org/entity/Q40523,Julia Roberts,Academy Award for Best Actress
4,http://www.wikidata.org/entity/Q40523,Julia Roberts,Golden Globe Award for Best Actress – Motion P...


In [389]:
df_awards1 = DataFrame({'num_awards' : df_awards.groupby( [ "human"] ).size()}).reset_index()

In [390]:
df_awards1.head()

,human,num_awards
0,http://www.wikidata.org/entity/Q1001214,2
1,http://www.wikidata.org/entity/Q100440,2
2,http://www.wikidata.org/entity/Q100937,60
3,http://www.wikidata.org/entity/Q101797,4
4,http://www.wikidata.org/entity/Q1018606,1


In [391]:
df_base = pd.merge(df_base,
                 df_awards1[['human', 'num_awards']],
                 on='human', how='left')

In [392]:
df_base.head()

,humanLabel,human,countryname,gendername,father_actor,mother_actor,sibling_actor,father_singer,mother_singer,sibling_singer,is_actor,is_singer,num_awards
0,Elvis Presley,http://www.wikidata.org/entity/Q303,American,Male,1,0,0,0,0,0,1,1,12.0
1,Bob Dylan,http://www.wikidata.org/entity/Q392,American,Male,0,0,0,0,0,0,1,1,60.0
2,Patti Smith,http://www.wikidata.org/entity/Q557,American,Female,0,0,0,0,0,0,0,1,5.0
3,Neil Young,http://www.wikidata.org/entity/Q633,American,Male,0,0,0,0,0,1,1,1,45.0
4,Stevie Wonder,http://www.wikidata.org/entity/Q714,American,Male,0,0,0,0,1,0,0,1,8.0


In [393]:
rq5 = """
SELECT DISTINCT ?human ?film

WHERE
{
?human wdt:P31 wd:Q5

{ { ?human wdt:P106 wd:Q10800557 } UNION {?human wdt:P106 wd:Q33999} } .

{ { ?human wdt:P27 wd:Q30 } UNION {?human wdt:P27 wd:Q668 } } .

?film wdt:P31 wd:Q11424 ; wdt:P161 ?human

SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
}
"""
df_movies = get_sparql_dataframe(wds, rq5)
df_movies.head()

,human,film
0,http://www.wikidata.org/entity/Q330315,http://www.wikidata.org/entity/Q46918
1,http://www.wikidata.org/entity/Q184885,http://www.wikidata.org/entity/Q46918
2,http://www.wikidata.org/entity/Q252290,http://www.wikidata.org/entity/Q46918
3,http://www.wikidata.org/entity/Q95043,http://www.wikidata.org/entity/Q47703
4,http://www.wikidata.org/entity/Q108622,http://www.wikidata.org/entity/Q47703


In [394]:
df_movies1 = DataFrame({'num_movies' : df_movies.groupby([ "human"] ).size()}).reset_index()

In [395]:
df_base = pd.merge(df_base,
                 df_movies1[['human', 'num_movies']],
                 on='human', how='left')

In [396]:
rq6 = """
SELECT DISTINCT ?human ?album ?single 

WHERE
{
?human wdt:P31 wd:Q5

{ { ?human wdt:P27 wd:Q30 } UNION {?human wdt:P27 wd:Q668 } } .
?human wdt:P106 wd:Q177220 .

OPTIONAL {?album wdt:P31 wd:Q482994 ; wdt:P175 ?human .}
OPTIONAL {?single wdt:P175 ?human .}
       
SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
}
"""
df_albums = get_sparql_dataframe(wds, rq6)
df_albums.head()

,human,album,single
0,http://www.wikidata.org/entity/Q71588,http://www.wikidata.org/entity/Q1259074,http://www.wikidata.org/entity/Q1259074
1,http://www.wikidata.org/entity/Q71588,http://www.wikidata.org/entity/Q1259074,http://www.wikidata.org/entity/Q1536174
2,http://www.wikidata.org/entity/Q71588,http://www.wikidata.org/entity/Q1259074,http://www.wikidata.org/entity/Q1601336
3,http://www.wikidata.org/entity/Q71588,http://www.wikidata.org/entity/Q1259074,http://www.wikidata.org/entity/Q1704102
4,http://www.wikidata.org/entity/Q71588,http://www.wikidata.org/entity/Q1259074,http://www.wikidata.org/entity/Q1974356


In [397]:
df_albums1 = DataFrame({'num_albums' : df_albums.groupby(["human"]).album.nunique()}).reset_index()
df_base = pd.merge(df_base,
                 df_albums1[['human', 'num_albums']],
                 on='human', how='left')
df_albums2 = DataFrame({'num_single' : df_albums.groupby(["human"]).single.nunique()}).reset_index()
df_base = pd.merge(df_base,
                 df_albums2[['human', 'num_single']],
                 on='human', how='left')

In [398]:
df_base

,humanLabel,human,countryname,gendername,father_actor,mother_actor,sibling_actor,father_singer,mother_singer,sibling_singer,is_actor,is_singer,num_awards,num_movies,num_albums,num_single
0,Elvis Presley,http://www.wikidata.org/entity/Q303,American,Male,1,0,0,0,0,0,1,1,12.0,39.0,229.0,655.0
1,Bob Dylan,http://www.wikidata.org/entity/Q392,American,Male,0,0,0,0,0,0,1,1,60.0,11.0,41.0,557.0
2,Patti Smith,http://www.wikidata.org/entity/Q557,American,Female,0,0,0,0,0,0,0,1,5.0,NaN,11.0,39.0
3,Neil Young,http://www.wikidata.org/entity/Q633,American,Male,0,0,0,0,0,1,1,1,45.0,13.0,2.0,204.0
4,Stevie Wonder,http://www.wikidata.org/entity/Q714,American,Male,0,0,0,0,1,0,0,1,8.0,NaN,22.0,124.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51973,Malvina Pray Florence,http://www.wikidata.org/entity/Q97300477,American,Female,0,0,1,0,0,0,1,0,NaN,NaN,NaN,NaN
51974,Maggie Newton,http://www.wikidata.org/entity/Q97302021,American,Female,0,0,0,0,0,0,1,0,NaN,NaN,NaN,NaN
51975,Zion Moreno,http://www.wikidata.org/entity/Q97302631,American,Unknown,0,0,0,0,0,0,1,0,NaN,NaN,NaN,NaN
51976,Maria Pray Williams,http://www.wikidata.org/entity/Q97303420,American,Female,0,0,1,0,0,0,1,0,NaN,NaN,NaN,NaN


In [399]:
rq7 = """
SELECT ?human ?humanLabel ?educationLabel ?educationcatLabel

WHERE

{
?human wdt:P31 wd:Q5

{ { ?human wdt:P106 wd:Q10800557 } UNION {?human wdt:P106 wd:Q33999} UNION {?human wdt:P106 wd:Q177220} } .

{ { ?human wdt:P27 wd:Q30 } UNION {?human wdt:P27 wd:Q668 } } .

?human wdt:P69 ?education .
?education wdt:P31 ?educationcat .

SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
}
"""
df_edu = get_sparql_dataframe(wds, rq7)
df_edu.head()

,human,humanLabel,educationLabel,educationcatLabel
0,http://www.wikidata.org/entity/Q4227,James Coburn,University of Southern California,private university
1,http://www.wikidata.org/entity/Q4461,Troian Bellisario,University of Southern California,private university
2,http://www.wikidata.org/entity/Q5104,Miranda Cosgrove,University of Southern California,private university
3,http://www.wikidata.org/entity/Q4227,James Coburn,University of Southern California,research university
4,http://www.wikidata.org/entity/Q4461,Troian Bellisario,University of Southern California,research university


In [400]:
lst_act_school = ['film', 'drama school', 'film school', 'theatre', 'theatrical troupe', 'theatre troupe', 
                  'performing arts school', 
                  'theatre festival', 'television series', 'film genre', 'dance studio', 'ballet school', 
                  'improvisational theatre',
                  'dance troupe']
lst_mus_school = ['music', 'performing arts school', 'college of music', 'music school', "children's music school", 
                  'music festival',
                  'musical ensemble']

In [401]:
df_base = df_base.assign(is_act_school=df_base.human.isin(df_edu[df_edu['educationcatLabel'].isin(lst_act_school)].human).astype(int))

In [402]:
df_base = df_base.assign(is_mus_school=df_base.human.isin(df_edu[df_edu['educationcatLabel'].isin(lst_mus_school)].human).astype(int))

In [403]:
df_base.to_csv('base.csv')